# Single particle tracking analysis

In [ ]:
import os
import pickle
import sys
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import proplot as pplt
from ipywidgets import interact
from ipywidgets import widgets

sys.path.append("/home/46h/repo/psdist/")
import psdist as ps
import psdist.visualization as psv

In [ ]:
pplt.rc["cycle"] = "538"
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["figure.facecolor"] = "white"
pplt.rc["grid"] = False

## Load data 

In [ ]:
timestamp = 230914140229
datadir = f"../../data_output/sns_ring/track_particle/{timestamp}"
os.listdir(datadir)

## Linear transfer matrix analysis

### Uncoupled

In [ ]:
file = open(os.path.join(datadir, "lattice_params_uncoupled.pkl"), "rb")
lattice_params =  pickle.load(file, encoding="latin1")
file.close()

pprint(lattice_params)

In [ ]:
filename = os.path.join(datadir, "lattice_twiss.dat")
twiss = pd.read_table(filename, sep=" ", index_col=0)
twiss

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2))
ax.plot(twiss["s"], twiss["beta_x"], label=r"$\beta_x$")
ax.plot(twiss["s"], twiss["beta_y"], label=r"$\beta_y$")
ax.legend(loc="right", ncols=1)
ax.format(xlabel="Position [m]", ylabel="[m/rad]")

### Coupled

In [ ]:
file = open(os.path.join(datadir, "lattice_params_coupled.pkl"), "rb")
lattice_params_coupled = pickle.load(file, encoding="latin1")
file.close()

pprint(lattice_params_coupled)

These might be wrong.

In [ ]:
filename = os.path.join(datadir, "lattice_twiss_coupled.dat")
twiss_coupled = pd.read_table(filename, sep=" ", index_col=0)
twiss_coupled

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2))
_colors = pplt.Cycle("538").by_key()["color"]
ax.plot(twiss_coupled["s"], twiss_coupled["beta_1x"], label=r"$\beta_{1x}$", color=_colors[0])
ax.plot(twiss_coupled["s"], twiss_coupled["beta_1y"], label=r"$\beta_{2y}$", color=_colors[0], alpha=0.2)
ax.plot(twiss_coupled["s"], twiss_coupled["beta_2x"], label=r"$\beta_{1x}$", color=_colors[1], alpha=0.2)
ax.plot(twiss_coupled["s"], twiss_coupled["beta_2y"], label=r"$\beta_{2y}$", color=_colors[1])
ax.legend(loc="right", ncols=1)
ax.format(xlabel="Position [m]", ylabel="[m/rad]")

## Turn-by-turn coordinates 

In [ ]:
os.listdir(datadir)

In [ ]:
coords = np.loadtxt(os.path.join(datadir, "coords.dat"))
coords[:, :4] *= 1000.0  # [m, rad] -> [mm, mrad]
coords[:, 5] *= 1000.0  # [MeV] -> [GeV]
coords.shape

In [ ]:
dims = ["x", "xp", "y", "yp", "z", "dE"]
units = ["mm", "mrad", "mm", "m", "m", "MeV"]
dims_units = [f"{dim} [{unit}]" for dim, unit in zip(dims, units)]

In [ ]:
@interact(
    dim1=widgets.Dropdown(options=dims, value="x"),
    dim2=widgets.Dropdown(options=dims, value="y"),
)
def update(dim1, dim2):
    axis = [dims.index(dim1), dims.index(dim2)]
    fig, ax = pplt.subplots()
    ax.scatter(coords[:, axis[0]], coords[:, axis[1]], s=5, color="black")
    ax.format(xlabel=dims_units[axis[0]], ylabel=dims_units[axis[1]])
    pplt.show()

In [ ]:
axis = (0, 1, 2, 3, 4, 5)
d = len(axis)
grid = psv.CornerGrid(d=d, diag=False, labels=dims_units)
grid.plot_cloud(coords[:, axis], kind="scatter", autolim_kws=dict(pad=0.1))